In [1]:
from datetime import date
import requests
from typing import Dict

from loguru import logger
import polars as pl

today = date.today().strftime("%Y-%m-%d")

# GTFS DATA

In [ ]:
def get_at_gtfs_data_from_at_mobile_api(
    data_name: str,
    params: Dict[str, str | int] = {},
    headers: Dict[str, str | int] = {}
) -> pl.DataFrame:
    
    url = f"https://api.at.govt.nz/gtfs/v3/{data_name}"
    
    response = requests.get(
        url,
        params=params, 
        headers=headers
    )
    
    # logger.info(f"Request URL: {response.url}")
    if response.status_code != 200:
        logger.error(f"Request failed with status code {response.status_code}: {response.text}")
    
    data = response.json()
    if "data" not in data:
        logger.error("Expected 'data' key in the JSON response")
    
    json_data = data["data"]

    # Load data into Polars DataFrame
    df = pl.DataFrame(json_data)
    
    for col, dtype in df.schema.items():
        if isinstance(dtype, pl.datatypes.Struct):
            df = df.unnest(col)
            
    return df
    

## Stops data

In [3]:

df_stops = get_at_gtfs_data_from_at_mobile_api(
    data_name="stops",
    params={
        "filter[date]": today
    },
    headers = {
        "Cache-Control": "no-cache",
        "Ocp-Apim-Subscription-Key": "5d22b4c49b284b9b8ace5d1971819052"
    }
)
display(df_stops)

type,id,location_type,parent_station,platform_code,stop_code,stop_id,stop_lat,stop_lon,stop_name,wheelchair_boarding
str,str,i64,str,str,str,str,f64,f64,str,i64
"""stop""","""100-56c57897""",1,null,null,"""100""","""100-56c57897""",-36.97766,174.84925,"""Papatoetoe Train Station""",0
"""stop""","""1002-c8bb8209""",0,"""31986-b1e95b83""","""A""","""1002""","""1002-c8bb8209""",-36.84316,174.76576,"""Stop A Lower Albert""",0
"""stop""","""1003-ea94d2b2""",0,"""31986-b1e95b83""","""B""","""1003""","""1003-ea94d2b2""",-36.84381,174.76551,"""Stop B Lower Albert""",0
"""stop""","""10036-288916d9""",1,null,null,"""10036""","""10036-288916d9""",-36.89726,174.80712,"""Ellerslie""",0
"""stop""","""1004-42737000""",0,"""31986-b1e95b83""","""C""","""1004""","""1004-42737000""",-36.84403,174.7655,"""Stop C Lower Albert""",0
…,…,…,…,…,…,…,…,…,…,…
"""stop""","""9925-fa8611dc""",0,null,null,"""9925""","""9925-fa8611dc""",-36.85073,174.75679,"""Sam Wrigley Street/ Cook Stree…",0
"""stop""","""9974-6b1dd4cc""",0,null,null,"""9974""","""9974-6b1dd4cc""",-36.75943,174.74654,"""Madison Place""",0
"""stop""","""9975-9f7b8faa""",0,null,null,"""9975""","""9975-9f7b8faa""",-36.75716,174.74401,"""Linwood Avenue""",0


In [4]:
df_stops.filter(pl.col("stop_name") == "Pitt Street").select(
    "stop_id",
    "stop_code",
).to_dict(as_series=False)

{'stop_id': ['7130-50c017db', '7133-173774ea'], 'stop_code': ['7130', '7133']}

In [5]:
df_stops.filter(pl.col("stop_name") == "Pitt Street")["id"].to_list()[0]

'7130-50c017db'

In [6]:
display(df_stops.filter(pl.col("stop_name") == "Bullock Track"))

type,id,location_type,parent_station,platform_code,stop_code,stop_id,stop_lat,stop_lon,stop_name,wheelchair_boarding
str,str,i64,str,str,str,str,f64,f64,str,i64
"""stop""","""8119-e41c6252""",0,null,null,"""8119""","""8119-e41c6252""",-36.86614,174.73094,"""Bullock Track""",0
"""stop""","""8120-1485cc06""",0,null,null,"""8120""","""8120-1485cc06""",-36.86631,174.73101,"""Bullock Track""",0


## Routes data

In [7]:
df_routes = get_at_gtfs_data_from_at_mobile_api(
    data_name="routes",
    headers = {
        "Cache-Control": "no-cache",
        "Ocp-Apim-Subscription-Key": "5d22b4c49b284b9b8ace5d1971819052"
    }
)

In [8]:
display(df_routes)

type,id,agency_id,route_color,route_id,route_long_name,route_short_name,route_text_color,route_type
str,str,str,str,str,str,str,str,i64
"""route""","""101-202""","""NZB""",null,"""101-202""","""101""","""101""",null,3
"""route""","""186-203""","""RTH""",null,"""186-203""","""186""","""186""",null,3
"""route""","""191-203""","""RTH""",null,"""191-203""","""191""","""191""",null,3
"""route""","""195-203""","""RTH""",null,"""195-203""","""195""","""195""",null,3
"""route""","""WX1-207""","""TZG""",null,"""WX1-207""","""WX1""","""WX1""",null,3
…,…,…,…,…,…,…,…,…
"""route""","""S813-202""","""NZB""",null,"""S813-202""","""S813""","""813""",null,712
"""route""","""STH-201""","""AM""","""D52923""","""STH-201""","""STH""","""STH""","""FFFFFF""",2
"""route""","""TMK-202""","""NZB""","""007AB8""","""TMK-202""","""TMK""","""TMK""","""FFFFFF""",3


In [9]:
display(df_routes.filter(pl.col("route_short_name") == "18"))

type,id,agency_id,route_color,route_id,route_long_name,route_short_name,route_text_color,route_type
str,str,str,str,str,str,str,str,i64
"""route""","""18-202""","""NZB""",null,"""18-202""","""18""","""18""",null,3


## Trips by route ID

In [10]:
df_trips_by_route = get_at_gtfs_data_from_at_mobile_api(
    data_name="routes/S025-203/trips",
    headers = {
        "Cache-Control": "no-cache",
        "Ocp-Apim-Subscription-Key": "5d22b4c49b284b9b8ace5d1971819052"
    }
)

In [11]:
display(df_trips_by_route)

type,id,bikes_allowed,direction_id,route_id,service_id,shape_id,trip_headsign,trip_id,wheelchair_accessible
str,str,i64,i64,str,str,str,str,str,i64
"""trip""","""275-02568-55500-2-32122e7b""",0,1,"""S025-203""","""Weekday-4""","""275-02568-27dcc78e""","""Westlake Schools To Torbay""","""275-02568-55500-2-32122e7b""",0


## Stoptimes by trip ID

In [14]:
df_stopstimes_by_trip_id = get_at_gtfs_data_from_at_mobile_api(
    data_name="trips/1153-07005-51600-2-e7801de1/stoptimes",
    headers = {
        "Cache-Control": "no-cache",
        "Ocp-Apim-Subscription-Key": "5d22b4c49b284b9b8ace5d1971819052"
    }
)

In [15]:
display(
    df_stopstimes_by_trip_id
    .join(
        df_stops.select("id", "stop_name"),
        left_on="stop_id",
        right_on="id",
        how="left"
    )
)

type,id,arrival_time,departure_time,drop_off_type,pickup_type,shape_dist_traveled,stop_headsign,stop_id,stop_sequence,timepoint,trip_id,stop_name
str,str,str,str,i64,i64,f64,str,str,i64,i64,str,str
"""stoptime""","""1153-07005-51600-2-e7801de1_1""","""14:20:00""","""14:20:00""",0,0,0.0,"""BRITOMART""","""6231-190b2aaa""",1,0,"""1153-07005-51600-2-e7801de1""","""Stop A Botany Town Centre"""
"""stoptime""","""1153-07005-51600-2-e7801de1_2""","""14:21:11""","""14:21:11""",0,0,0.503,"""BRITOMART""","""6294-9919bdf3""",2,0,"""1153-07005-51600-2-e7801de1""","""Woolworths Botany"""
"""stoptime""","""1153-07005-51600-2-e7801de1_3""","""14:22:05""","""14:22:05""",0,0,0.885,"""BRITOMART""","""6296-473fe125""",3,0,"""1153-07005-51600-2-e7801de1""","""The Hub"""
"""stoptime""","""1153-07005-51600-2-e7801de1_4""","""14:23:28""","""14:23:28""",0,0,1.478,"""BRITOMART""","""6137-efe1e5a5""",4,0,"""1153-07005-51600-2-e7801de1""","""Burswood Drive"""
"""stoptime""","""1153-07005-51600-2-e7801de1_5""","""14:24:31""","""14:24:31""",0,0,1.927,"""BRITOMART""","""6139-f2088f38""",5,0,"""1153-07005-51600-2-e7801de1""","""Harris Road"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""stoptime""","""1153-07005-51600-2-e7801de1_39""","""15:22:34""","""15:22:34""",0,0,18.736,"""BRITOMART""","""7149-6d6d1e99""",39,0,"""1153-07005-51600-2-e7801de1""","""Symonds Street/Karangahape Roa…"
"""stoptime""","""1153-07005-51600-2-e7801de1_40""","""15:24:48""","""15:24:48""",0,0,19.215,"""BRITOMART""","""7147-4e9003b4""",40,0,"""1153-07005-51600-2-e7801de1""","""Stop E Auckland Universities"""
"""stoptime""","""1153-07005-51600-2-e7801de1_41""","""15:27:15""","""15:27:15""",0,0,19.739,"""BRITOMART""","""7145-caabaedd""",41,0,"""1153-07005-51600-2-e7801de1""","""Waterloo Quadrant"""


## Stop trips by Stop id

In [16]:
df_stop_trips_by_stop_id = get_at_gtfs_data_from_at_mobile_api(
    data_name="stops/7133-173774ea/stoptrips",
    params={
        "filter[date]": "2025-06-15",
        "filter[start_hour]": 7,
        "filter[hour_range]": 15
    },
    headers = {
        "Cache-Control": "no-cache",
        "Ocp-Apim-Subscription-Key": "5d22b4c49b284b9b8ace5d1971819052"
    }
)
# https://api.at.govt.nz/gtfs/v3/stops/8119-e41c6252/stoptrips?filter[date]=2025-06-14&filter[start_hour]=8&filter%5Bhour_range%5D=5

# df_stop_trips_by_stop_id["route_id"].unique()

display(df_stop_trips_by_stop_id)

type,id,arrival_time,departure_time,direction_id,drop_off_type,pickup_type,route_id,service_date,shape_id,stop_headsign,stop_id,stop_sequence,trip_headsign,trip_id,trip_start_time
str,str,str,str,i64,i64,i64,str,str,str,str,str,i64,str,str,str
"""stoptrip""","""stop:7133-173774ea_trip:1222-0…","""07:08:11""","""07:08:11""",0,0,0,"""11W-207""","""2025-06-15""","""1222-01107-30fca653""","""CITY CENTRE""","""7133-173774ea""",34,"""Northwest To City Centre Via W…","""1222-01107-23160-2-c7765123""","""06:26:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1292-0…","""07:12:51""","""07:12:51""",0,0,0,"""WX1-207""","""2025-06-15""","""1292-01001-aac3cb89""","""CITY CENTRE""","""7133-173774ea""",6,"""Northwest To City Centre Via M…","""1292-01001-24300-2-8e3771d4""","""06:45:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1075-0…","""07:13:17""","""07:13:17""",0,0,0,"""18-202""","""2025-06-15""","""1075-01803-88e0122a""","""CITY CENTRE""","""7133-173774ea""",27,"""New Lynn To City Centre Via Gr…","""1075-01803-24300-2-d1ba66eb""","""06:45:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1203-0…","""07:25:49""","""07:25:49""",0,0,0,"""11T-207""","""2025-06-15""","""1203-01105-c0cbff53""","""CITY CENTRE""","""7133-173774ea""",29,"""Northwest To City Centre Via T…","""1203-01105-24300-2-77683a6f""","""06:45:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1292-0…","""07:27:51""","""07:27:51""",0,0,0,"""WX1-207""","""2025-06-15""","""1292-01001-aac3cb89""","""CITY CENTRE""","""7133-173774ea""",6,"""Northwest To City Centre Via M…","""1292-01001-25200-2-8e3771d4""","""07:00:00"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""stoptrip""","""stop:7133-173774ea_trip:1292-0…","""21:36:47""","""21:36:47""",0,0,0,"""WX1-207""","""2025-06-15""","""1292-01001-aac3cb89""","""CITY CENTRE""","""7133-173774ea""",6,"""Northwest To City Centre Via M…","""1292-01001-75600-2-9b98cbc4""","""21:00:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1222-0…","""21:38:11""","""21:38:11""",0,0,0,"""11W-207""","""2025-06-15""","""1222-01107-30fca653""","""CITY CENTRE""","""7133-173774ea""",34,"""Northwest To City Centre Via W…","""1222-01107-75360-2-c7765123""","""20:56:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1075-0…","""21:45:56""","""21:45:56""",0,0,0,"""18-202""","""2025-06-15""","""1075-01803-88e0122a""","""CITY CENTRE""","""7133-173774ea""",27,"""New Lynn To City Centre Via Gr…","""1075-01803-76500-2-975633d0""","""21:15:00"""


## Calendar per Service ID

In [17]:
service_IDs = df_trips_by_route.select("service_id").unique().to_series().to_list()

dfs = []

for service_id in service_IDs:
    df_calendars = get_at_gtfs_data_from_at_mobile_api(
        data_name=f"services/{service_id}/calendars",
        headers = {
            "Cache-Control": "no-cache",
            "Ocp-Apim-Subscription-Key": "5d22b4c49b284b9b8ace5d1971819052"
        }
    )
    dfs.append(df_calendars)
    
df_calendars_by_service = pl.concat(dfs)

display(df_calendars_by_service)

type,id,end_date,friday,monday,saturday,service_id,start_date,sunday,thursday,tuesday,wednesday
str,str,str,i64,i64,i64,str,str,i64,i64,i64,i64
"""calendar""","""Weekday-4""","""2025-09-28""",1,1,0,"""Weekday-4""","""2025-06-09""",0,1,1,1


# extract Avondale -> Sagrado Cantina

In [18]:
stop_codes = [
    "8147",
    "8545",
    "7149",
    "8331",
    "7133"
]

df_stops_av_sc = (
    df_stops
    .filter(
        pl.col("stop_code")
        .is_in(stop_codes)
    )
)
display(df_stops_av_sc)

type,id,location_type,parent_station,platform_code,stop_code,stop_id,stop_lat,stop_lon,stop_name,wheelchair_boarding
str,str,i64,str,str,str,str,f64,f64,str,i64
"""stop""","""7133-173774ea""",0,null,null,"""7133""","""7133-173774ea""",-36.85773,174.75968,"""Pitt Street""",0
"""stop""","""7149-6d6d1e99""",0,null,null,"""7149""","""7149-6d6d1e99""",-36.85741,174.76449,"""Symonds Street/Karangahape Roa…",0
"""stop""","""8147-58c872f4""",0,null,null,"""8147""","""8147-58c872f4""",-36.89337,174.69607,"""Avondale Police Station""",0
"""stop""","""8331-c2dad3f5""",0,null,null,"""8331""","""8331-c2dad3f5""",-36.89569,174.70094,"""Avondale Islamic Centre""",0
"""stop""","""8545-aed7c410""",0,null,null,"""8545""","""8545-aed7c410""",-36.89247,174.68973,"""Rosebank Road/Victor Street""",0


In [19]:
df_stops_av_sc["id"].to_list()

['7133-173774ea',
 '7149-6d6d1e99',
 '8147-58c872f4',
 '8331-c2dad3f5',
 '8545-aed7c410']

In [ ]:
stop_ids = df_stops_av_sc["id"].to_list()
dates = [
    "2025-06-09",
    "2025-06-10",
    "2025-06-11",
    "2025-06-12",
    "2025-06-13",
    "2025-06-14",
    "2025-06-15",
    "2025-06-16"
]

trips_av_sc = []

for stop_id in stop_ids:
    for api_date in dates:
        logger.info(f"Fetching trips for stop {stop_id} on date {api_date}")
        df_trips = get_at_gtfs_data_from_at_mobile_api(
            data_name=f"stops/{stop_id}/stoptrips",
            params={
                "filter[date]": date,
                "filter[start_hour]": 3,
                "filter[hour_range]": 24
            },
            headers = {
                "Cache-Control": "no-cache",
                "Ocp-Apim-Subscription-Key": "5d22b4c49b284b9b8ace5d1971819052"
            }
        )
        trips_av_sc.append(df_trips)
    
df_trips_av_sc_api = pl.concat(trips_av_sc)
display(df_trips_av_sc_api)

2025-06-18 13:27:23.167 | INFO     | __main__:<module>:17 - Fetching trips for stop 7133-173774ea on date 2025-06-09
2025-06-18 13:27:23.605 | INFO     | __main__:<module>:17 - Fetching trips for stop 7133-173774ea on date 2025-06-10
2025-06-18 13:27:23.953 | INFO     | __main__:<module>:17 - Fetching trips for stop 7133-173774ea on date 2025-06-11
2025-06-18 13:27:24.314 | INFO     | __main__:<module>:17 - Fetching trips for stop 7133-173774ea on date 2025-06-12
2025-06-18 13:27:24.656 | INFO     | __main__:<module>:17 - Fetching trips for stop 7133-173774ea on date 2025-06-13
2025-06-18 13:27:25.006 | INFO     | __main__:<module>:17 - Fetching trips for stop 7133-173774ea on date 2025-06-14
2025-06-18 13:27:25.345 | INFO     | __main__:<module>:17 - Fetching trips for stop 7133-173774ea on date 2025-06-15
2025-06-18 13:27:25.674 | INFO     | __main__:<module>:17 - Fetching trips for stop 7133-173774ea on date 2025-06-16
2025-06-18 13:27:26.184 | INFO     | __main__:<module>:17 - Fetc

type,id,arrival_time,departure_time,direction_id,drop_off_type,pickup_type,route_id,service_date,shape_id,stop_headsign,stop_id,stop_sequence,trip_headsign,trip_id,trip_start_time
str,str,str,str,i64,i64,i64,str,str,str,str,str,i64,str,str,str
"""stoptrip""","""stop:7133-173774ea_trip:1075-0…","""05:26:31""","""05:26:31""",0,0,0,"""18-202""","""2025-06-09""","""1075-01803-c51eb07f""","""CITY CENTRE""","""7133-173774ea""",28,"""New Lynn To City Centre Via Gr…","""1075-01803-18000-2-a74f27ae""","""05:00:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1292-0…","""05:27:51""","""05:27:51""",0,0,0,"""WX1-207""","""2025-06-09""","""1292-01001-aac3cb89""","""CITY CENTRE""","""7133-173774ea""",6,"""Northwest To City Centre Via M…","""1292-01001-18000-2-8e3771d4""","""05:00:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1075-0…","""05:41:31""","""05:41:31""",0,0,0,"""18-202""","""2025-06-09""","""1075-01803-c51eb07f""","""CITY CENTRE""","""7133-173774ea""",28,"""New Lynn To City Centre Via Gr…","""1075-01803-18900-2-a74f27ae""","""05:15:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1222-0…","""05:50:33""","""05:50:33""",0,0,0,"""11W-207""","""2025-06-09""","""1222-01107-a49f22fb""","""CITY CENTRE""","""7133-173774ea""",35,"""Northwest To City Centre Via W…","""1222-01107-18480-2-3f5e175f""","""05:08:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1095-1…","""05:56:33""","""05:56:33""",0,0,0,"""195-203""","""2025-06-09""","""1095-19507-b83a1774""","""CITY CENTRE""","""7133-173774ea""",57,"""New Lynn To City Centre Via Gr…","""1095-19507-18900-2-f1cd1a53""","""05:15:00"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""stoptrip""","""stop:8545-aed7c410_trip:1278-0…","""20:26:26""","""20:26:26""",0,0,0,"""22R-202""","""2025-06-16""","""1278-02201-e82e4758""","""CITY CENTRE""","""8545-aed7c410""",12,"""Rosebank Rd To City Centre Via…","""1278-02201-73260-2-876d2346""","""20:21:00"""
"""stoptrip""","""stop:8545-aed7c410_trip:1278-0…","""20:56:26""","""20:56:26""",0,0,0,"""22R-202""","""2025-06-16""","""1278-02201-e82e4758""","""CITY CENTRE""","""8545-aed7c410""",12,"""Rosebank Rd To City Centre Via…","""1278-02201-75060-2-876d2346""","""20:51:00"""
"""stoptrip""","""stop:8545-aed7c410_trip:1278-0…","""21:26:26""","""21:26:26""",0,0,0,"""22R-202""","""2025-06-16""","""1278-02201-e82e4758""","""CITY CENTRE""","""8545-aed7c410""",12,"""Rosebank Rd To City Centre Via…","""1278-02201-76860-2-876d2346""","""21:21:00"""


In [ ]:
df_stop_date1 = get_at_gtfs_data_from_at_mobile_api(
        data_name="stops/7133-173774ea/stoptrips",
        params={
            "filter[date]": "2025-06-13",
            "filter[start_hour]": 3,
            "filter[hour_range]": 24
        },
        headers = {
            "Cache-Control": "no-cache",
            "Ocp-Apim-Subscription-Key": "5d22b4c49b284b9b8ace5d1971819052"
        }
    ).sort("arrival_time")

df_stop_date2 = get_at_gtfs_data_from_at_mobile_api(
        data_name="stops/7133-173774ea/stoptrips",
        params={
            "filter[date]": "2025-06-16",
            "filter[start_hour]": 3,
            "filter[hour_range]": 24
        },
        headers = {
            "Cache-Control": "no-cache",
            "Ocp-Apim-Subscription-Key": "5d22b4c49b284b9b8ace5d1971819052"
        }
    ).sort("arrival_time")




In [70]:
df_stop_date1 == df_stop_date2

type,id,arrival_time,departure_time,direction_id,drop_off_type,pickup_type,route_id,service_date,shape_id,stop_headsign,stop_id,stop_sequence,trip_headsign,trip_id,trip_start_time
bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool
true,false,true,true,true,true,true,true,false,false,true,true,false,true,false,true
true,true,true,true,true,true,true,true,false,true,true,true,true,true,true,true
true,false,true,true,true,true,true,true,false,false,true,true,false,true,false,true
true,false,false,false,true,true,true,true,false,false,true,true,false,true,false,true
true,true,true,true,true,true,true,true,false,true,true,true,true,true,true,true
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
true,false,false,false,true,true,true,false,false,false,true,true,false,false,false,false
true,false,false,false,true,true,true,false,false,false,true,true,false,false,false,false
true,false,true,true,true,true,true,true,false,false,true,true,false,true,false,true


In [71]:
df_stop_date1

type,id,arrival_time,departure_time,direction_id,drop_off_type,pickup_type,route_id,service_date,shape_id,stop_headsign,stop_id,stop_sequence,trip_headsign,trip_id,trip_start_time
str,str,str,str,i64,i64,i64,str,str,str,str,str,i64,str,str,str
"""stoptrip""","""stop:7133-173774ea_trip:1075-0…","""05:26:31""","""05:26:31""",0,0,0,"""18-202""","""2025-06-13""","""1075-01803-c51eb07f""","""CITY CENTRE""","""7133-173774ea""",28,"""New Lynn To City Centre Via Gr…","""1075-01803-18000-2-a74f27ae""","""05:00:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1292-0…","""05:27:51""","""05:27:51""",0,0,0,"""WX1-207""","""2025-06-13""","""1292-01001-aac3cb89""","""CITY CENTRE""","""7133-173774ea""",6,"""Northwest To City Centre Via M…","""1292-01001-18000-2-8e3771d4""","""05:00:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1075-0…","""05:41:31""","""05:41:31""",0,0,0,"""18-202""","""2025-06-13""","""1075-01803-c51eb07f""","""CITY CENTRE""","""7133-173774ea""",28,"""New Lynn To City Centre Via Gr…","""1075-01803-18900-2-a74f27ae""","""05:15:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1222-0…","""05:50:33""","""05:50:33""",0,0,0,"""11W-207""","""2025-06-13""","""1222-01107-a49f22fb""","""CITY CENTRE""","""7133-173774ea""",35,"""Northwest To City Centre Via W…","""1222-01107-18480-2-3f5e175f""","""05:08:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1095-1…","""05:56:33""","""05:56:33""",0,0,0,"""195-203""","""2025-06-13""","""1095-19507-b83a1774""","""CITY CENTRE""","""7133-173774ea""",57,"""New Lynn To City Centre Via Gr…","""1095-19507-18900-2-f1cd1a53""","""05:15:00"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""stoptrip""","""stop:7133-173774ea_trip:1292-0…","""23:42:51""","""23:42:51""",0,0,0,"""WX1-207""","""2025-06-13""","""1292-01001-aac3cb89""","""CITY CENTRE""","""7133-173774ea""",6,"""Northwest To City Centre Via M…","""1292-01001-83700-2-8e3771d4""","""23:15:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1222-0…","""23:43:44""","""23:43:44""",0,0,0,"""11W-207""","""2025-06-13""","""1222-01107-a49f22fb""","""CITY CENTRE""","""7133-173774ea""",35,"""Northwest To City Centre Via W…","""1222-01107-82680-2-559d62ad""","""22:58:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1075-0…","""23:45:56""","""23:45:56""",0,0,0,"""18-202""","""2025-06-13""","""1075-01803-c51eb07f""","""CITY CENTRE""","""7133-173774ea""",28,"""New Lynn To City Centre Via Gr…","""1075-01803-83700-2-d62d3b73""","""23:15:00"""


In [72]:
df_stop_date2

type,id,arrival_time,departure_time,direction_id,drop_off_type,pickup_type,route_id,service_date,shape_id,stop_headsign,stop_id,stop_sequence,trip_headsign,trip_id,trip_start_time
str,str,str,str,i64,i64,i64,str,str,str,str,str,i64,str,str,str
"""stoptrip""","""stop:7133-173774ea_trip:1075-0…","""05:26:31""","""05:26:31""",0,0,0,"""18-202""","""2025-06-16""","""1075-01803-88e0122a""","""CITY CENTRE""","""7133-173774ea""",27,"""New Lynn To City Centre Via Gr…","""1075-01803-18000-2-0909bf84""","""05:00:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1292-0…","""05:27:51""","""05:27:51""",0,0,0,"""WX1-207""","""2025-06-16""","""1292-01001-aac3cb89""","""CITY CENTRE""","""7133-173774ea""",6,"""Northwest To City Centre Via M…","""1292-01001-18000-2-8e3771d4""","""05:00:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1075-0…","""05:41:31""","""05:41:31""",0,0,0,"""18-202""","""2025-06-16""","""1075-01803-88e0122a""","""CITY CENTRE""","""7133-173774ea""",27,"""New Lynn To City Centre Via Gr…","""1075-01803-18900-2-0909bf84""","""05:15:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1222-0…","""05:50:32""","""05:50:32""",0,0,0,"""11W-207""","""2025-06-16""","""1222-01107-30fca653""","""CITY CENTRE""","""7133-173774ea""",34,"""Northwest To City Centre Via W…","""1222-01107-18480-2-ed4752e5""","""05:08:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1095-1…","""05:56:33""","""05:56:33""",0,0,0,"""195-203""","""2025-06-16""","""1095-19507-b83a1774""","""CITY CENTRE""","""7133-173774ea""",57,"""New Lynn To City Centre Via Gr…","""1095-19507-18900-2-f1cd1a53""","""05:15:00"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""stoptrip""","""stop:7133-173774ea_trip:1222-0…","""23:40:11""","""23:40:11""",0,0,0,"""11W-207""","""2025-06-16""","""1222-01107-30fca653""","""CITY CENTRE""","""7133-173774ea""",34,"""Northwest To City Centre Via W…","""1222-01107-82680-2-2cc162c0""","""22:58:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1292-0…","""23:42:51""","""23:42:51""",0,0,0,"""WX1-207""","""2025-06-16""","""1292-01001-aac3cb89""","""CITY CENTRE""","""7133-173774ea""",6,"""Northwest To City Centre Via M…","""1292-01001-83700-2-8e3771d4""","""23:15:00"""
"""stoptrip""","""stop:7133-173774ea_trip:1075-0…","""23:45:56""","""23:45:56""",0,0,0,"""18-202""","""2025-06-16""","""1075-01803-88e0122a""","""CITY CENTRE""","""7133-173774ea""",27,"""New Lynn To City Centre Via Gr…","""1075-01803-83700-2-975633d0""","""23:15:00"""


In [36]:
# remove trip without 2 stops in the same trip_id
# It means that the trip doesn't start at avondale or doesn't end at sagrida cantina

df_trips_av_sc_api = (
    df_trips_av_sc_api
    .join(
        df_trips_av_sc_api
        .group_by("trip_id", "service_date", "route_id")
        .len()
        .filter(pl.col("len") > 1)
        .drop("len"),
        on=["trip_id", "service_date", "route_id"],
        how="inner"
    )
    .filter(pl.col("route_id") != "191-203")
)
df_trips_av_sc_api["route_id"].unique()


route_id
str
"""195-203"""
"""22R-202"""
"""18-202"""


In [50]:
df_trips_av_sc = (
    df_trips_av_sc_api
    .join(
        df_stops_av_sc.select("id", "stop_name"),
        left_on="stop_id",
        right_on="id",
        how="left"
    )
    .select("stop_id", "stop_name", "trip_id", "route_id", "trip_start_time", "service_date", "arrival_time")
    .sort(["trip_id", "service_date", "arrival_time"], descending=False)
    .with_columns(
        (
            (pl.col("service_date") + " " + pl.col("arrival_time"))
            .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S", strict=False)
            .alias("arrival_time")
        ),
        (
           pl.col("service_date")
           .str.strptime(pl.Date, "%Y-%m-%d")
        )          
    )
    
)
display(df_trips_av_sc)

stop_id,stop_name,trip_id,route_id,trip_start_time,service_date,arrival_time
str,str,str,str,str,date,datetime[μs]
"""8147-58c872f4""","""Avondale Police Station""","""1075-01803-18000-2-0909bf84""","""18-202""","""05:00:00""",2025-06-16,2025-06-16 05:06:37
"""7133-173774ea""","""Pitt Street""","""1075-01803-18000-2-0909bf84""","""18-202""","""05:00:00""",2025-06-16,2025-06-16 05:26:31
"""8147-58c872f4""","""Avondale Police Station""","""1075-01803-18000-2-a74f27ae""","""18-202""","""05:00:00""",2025-06-09,2025-06-09 05:06:37
"""7133-173774ea""","""Pitt Street""","""1075-01803-18000-2-a74f27ae""","""18-202""","""05:00:00""",2025-06-09,2025-06-09 05:26:31
"""8147-58c872f4""","""Avondale Police Station""","""1075-01803-18000-2-a74f27ae""","""18-202""","""05:00:00""",2025-06-10,2025-06-10 05:06:37
…,…,…,…,…,…,…
"""7149-6d6d1e99""","""Symonds Street/Karangahape Roa…","""1278-02201-81300-2-39bf8491""","""22R-202""","""22:35:00""",2025-06-16,2025-06-16 23:07:32
"""8545-aed7c410""","""Rosebank Road/Victor Street""","""1278-02201-84180-2-582be196""","""22R-202""","""23:23:00""",2025-06-14,2025-06-14 23:28:26
"""7149-6d6d1e99""","""Symonds Street/Karangahape Roa…","""1278-02201-84180-2-582be196""","""22R-202""","""23:23:00""",2025-06-14,2025-06-14 23:52:14


In [ ]:
df_trips_av_sc = (
    df_trips_av_sc
    .rename({
        "arrival_time": "start_datetime",
    })
    .with_columns(
        # Compute duration of trip from the starting bus stop to the final stop
        (
            pl.col("start_datetime")
            .diff()
            .over(["trip_id", "service_date"])
            .dt.total_minutes()
            .alias("trip_duration")
        ),
        # Get the final destination time in the same line as the starting stop
        (
            pl.col("start_datetime")
            .shift(-1)
            .over(["trip_id", "service_date"])
            .alias("arrival_datetime")
        )
    )
    .with_columns(
        # Get the trip duration in the line of the starting stop
        (
            pl.col("trip_duration")
            .shift(-1)
            .over(["trip_id", "service_date"])
        ).alias("trip_duration")
    )
    .drop_nulls(["trip_duration", "start_datetime", "arrival_datetime"])
    .sort(["start_datetime"], descending=False)
    .with_columns(
        (
            pl.col("start_datetime")
            .diff()
            .over(["route_id", "service_date"])
            .dt.total_minutes()
            .cast(pl.Int64)
            .alias("next_route_bus_waiting_time")
        ),
        (
            pl.col("start_datetime")
            .diff()
            .over(["service_date"])
            .dt.total_minutes()
            .cast(pl.Int64)
            .alias("next_bus_waiting_time")
        ),
        pl.col("service_date").dt.strftime("%A").alias("day_of_timeslot"),
        (
            pl.when(
                (pl.col("start_datetime").dt.hour() >= 11)
                & (pl.col("start_datetime").dt.hour() < 12)
            ).then(pl.lit("Start at 12"))
            .when(
                (pl.col("start_datetime").dt.hour() >= 10)
                & (pl.col("start_datetime").dt.hour() < 11)
            ).then(pl.lit("Start at 11"))
            .otherwise(pl.lit("Other"))
            .alias("time_slot")
        ),
        pl.col("start_datetime").dt.weekday().alias("weekday"),
        # Add walk time to the trip duration based on the route_id
        (
            pl.when(
                (pl.col("route_id") == "195-203")
            ).then(pl.col("trip_duration") + pl.lit(25))
            .when(
                (pl.col("route_id") == "22R-202")
            ).then(pl.col("trip_duration") + pl.lit(22))
            .when(
                (pl.col("route_id") == "18-202")
            ).then(pl.col("trip_duration") + pl.lit(17))
            .otherwise(pl.lit(-1))
            .alias("trip_duration")
        ),
    )
)

In [52]:
display(df_trips_av_sc)

stop_id,stop_name,trip_id,route_id,trip_start_time,service_date,start_datetime,trip_duration,arrival_datetime,next_route_bus_waiting_time,next_bus_waiting_time,day_of_timeslot,time_slot,weekday
str,str,str,str,str,date,datetime[μs],i64,datetime[μs],i64,i64,str,str,i8
"""8147-58c872f4""","""Avondale Police Station""","""1075-01803-18000-2-a74f27ae""","""18-202""","""05:00:00""",2025-06-09,2025-06-09 05:06:37,36,2025-06-09 05:26:31,null,null,"""Monday""","""Other""",1
"""8545-aed7c410""","""Rosebank Road/Victor Street""","""1278-02201-18120-2-e485a8f4""","""22R-202""","""05:02:00""",2025-06-09,2025-06-09 05:08:59,49,2025-06-09 05:36:32,null,2,"""Monday""","""Other""",1
"""8147-58c872f4""","""Avondale Police Station""","""1075-01803-18900-2-a74f27ae""","""18-202""","""05:15:00""",2025-06-09,2025-06-09 05:21:37,36,2025-06-09 05:41:31,15,12,"""Monday""","""Other""",1
"""8331-c2dad3f5""","""Avondale Islamic Centre""","""1095-19507-18900-2-f1cd1a53""","""195-203""","""05:15:00""",2025-06-09,2025-06-09 05:36:40,44,2025-06-09 05:56:33,null,15,"""Monday""","""Other""",1
"""8147-58c872f4""","""Avondale Police Station""","""1075-01803-19800-2-85f6d34b""","""18-202""","""05:30:00""",2025-06-09,2025-06-09 05:36:50,37,2025-06-09 05:57:24,15,0,"""Monday""","""Other""",1
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""8331-c2dad3f5""","""Avondale Islamic Centre""","""1095-19507-80100-2-06ff451d""","""195-203""","""22:15:00""",2025-06-16,2025-06-16 22:37:54,48,2025-06-16 23:01:00,29,0,"""Monday""","""Other""",1
"""8545-aed7c410""","""Rosebank Road/Victor Street""","""1278-02201-81300-2-39bf8491""","""22R-202""","""22:35:00""",2025-06-16,2025-06-16 22:40:26,49,2025-06-16 23:07:32,44,2,"""Monday""","""Other""",1
"""8147-58c872f4""","""Avondale Police Station""","""1075-01803-81900-2-975633d0""","""18-202""","""22:45:00""",2025-06-16,2025-06-16 22:52:43,40,2025-06-16 23:15:56,15,12,"""Monday""","""Other""",1


In [53]:
import plotly.express as px

fig = px.box(df_trips_av_sc, x="day_of_timeslot", y="trip_duration", color="route_id")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

In [41]:
import plotly.express as px

fig = px.box(df_trips_av_sc, x="day_of_timeslot", y="next_bus_waiting_time", color="route_id")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.show()

# Write in parquet file

In [73]:
df_trips_av_sc.write_parquet("data/at_gtfs_trips_av_sc.parquet")

In [54]:
df_trips_av_sc.write_csv("data/at_gtfs_trips_av_sc.csv")

In [29]:
display(pl.read_parquet("data/at_gtfs_trips_av_sc.parquet"))

stop_id,stop_name,trip_id,route_id,trip_start_time,service_date,start_datetime,trip_duration,arrival_datetime,next_route_bus_waiting_time,next_bus_waiting_time,day_of_timeslot,time_slot,weekday
str,str,str,str,str,date,datetime[μs],i64,datetime[μs],i64,i64,str,str,i8
"""8147-58c872f4""","""Avondale Police Station""","""1075-01803-18000-2-a74f27ae""","""18-202""","""05:00:00""",2025-06-09,2025-06-09 05:06:37,19,2025-06-09 05:26:31,null,null,"""Monday""","""Other""",1
"""8545-aed7c410""","""Rosebank Road/Victor Street""","""1278-02201-18120-2-e485a8f4""","""22R-202""","""05:02:00""",2025-06-09,2025-06-09 05:08:59,27,2025-06-09 05:36:32,null,2,"""Monday""","""Other""",1
"""8147-58c872f4""","""Avondale Police Station""","""1075-01803-18900-2-a74f27ae""","""18-202""","""05:15:00""",2025-06-09,2025-06-09 05:21:37,19,2025-06-09 05:41:31,15,12,"""Monday""","""Other""",1
"""8331-c2dad3f5""","""Avondale Islamic Centre""","""1095-19507-18900-2-f1cd1a53""","""195-203""","""05:15:00""",2025-06-09,2025-06-09 05:36:40,19,2025-06-09 05:56:33,null,15,"""Monday""","""Other""",1
"""8147-58c872f4""","""Avondale Police Station""","""1075-01803-19800-2-85f6d34b""","""18-202""","""05:30:00""",2025-06-09,2025-06-09 05:36:50,20,2025-06-09 05:57:24,15,0,"""Monday""","""Other""",1
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""8331-c2dad3f5""","""Avondale Islamic Centre""","""1095-19507-80100-2-06ff451d""","""195-203""","""22:15:00""",2025-06-16,2025-06-16 22:37:54,23,2025-06-16 23:01:00,29,0,"""Monday""","""Other""",1
"""8545-aed7c410""","""Rosebank Road/Victor Street""","""1278-02201-81300-2-39bf8491""","""22R-202""","""22:35:00""",2025-06-16,2025-06-16 22:40:26,27,2025-06-16 23:07:32,44,2,"""Monday""","""Other""",1
"""8147-58c872f4""","""Avondale Police Station""","""1075-01803-81900-2-975633d0""","""18-202""","""22:45:00""",2025-06-16,2025-06-16 22:52:43,23,2025-06-16 23:15:56,15,12,"""Monday""","""Other""",1


# Get data from cloud torage

In [74]:
from io import BytesIO
from google.cloud import storage
import polars as pl
from loguru import logger

In [75]:
def read_parquet_from_gcs(bucket_name: str, blob_name: str) -> pl.DataFrame:
    """
    Reads a Parquet file from Google Cloud Storage into a Polars DataFrame.

    Args:
        bucket_name: The name of the GCS bucket.
        blob_name: The path to the Parquet file within the bucket.

    Returns:
        A Polars DataFrame containing the data from the Parquet file.
    """
    try:
        client = storage.Client()  # Assumes GOOGLE_APPLICATION_CREDENTIALS is set
        bucket = client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Download the blob's content as bytes
        parquet_bytes = blob.download_as_bytes()

        # Read the Parquet data from the bytes
        df = pl.read_parquet(BytesIO(parquet_bytes))
        
        logger.info(f"Successfully read Parquet file '{blob_name}' from GCS bucket '{bucket_name}'.")
        return df
    except Exception as e:
        logger.error(f"Error reading Parquet file '{blob_name}' from GCS bucket '{bucket_name}': {e}")
        raise

In [ ]:
bucket_name = "at-bus-open-data"
    # Example blob name, replace with an actual blob path from your GCS
blob_name = "2025-06-17/trips_7149-6d6d1e99.parquet" 

try:
    df_from_gcs = read_parquet_from_gcs(bucket_name, blob_name)
    display(df_from_gcs)
except Exception as e:
    print(f"Failed to read data: {e}")

2025-06-18 17:52:24.481 | INFO     | __main__:read_parquet_from_gcs:23 - Successfully read Parquet file 'at-bus/2025-06-17/trips_7149-6d6d1e99.parquet' from GCS bucket 'pne-open-data'.


type,id,arrival_time,departure_time,direction_id,drop_off_type,pickup_type,route_id,service_date,shape_id,stop_headsign,stop_id,stop_sequence,trip_headsign,trip_id,trip_start_time,api_date_ingestion
str,str,str,str,i64,i64,i64,str,str,str,str,str,i64,str,str,str,date
"""stoptrip""","""stop:7149-6d6d1e99_trip:27-027…","""05:23:03""","""05:23:03""",0,0,0,"""27H-202""","""2025-06-17""","""27-02707-e4bba28d""","""BRITOMART""","""7149-6d6d1e99""",33,"""Waikowhai To Britomart Via Hil…","""27-02707-17700-2-2b57ccb3""","""04:55:00""",2025-06-17
"""stoptrip""","""stop:7149-6d6d1e99_trip:1283-0…","""05:34:08""","""05:34:08""",0,0,0,"""27W-202""","""2025-06-17""","""1283-02705-4ff7746f""","""BRITOMART""","""7149-6d6d1e99""",30,"""Waikowhai To Britomart Via Mt …","""1283-02705-18300-2-927d2e0c""","""05:05:00""",2025-06-17
"""stoptrip""","""stop:7149-6d6d1e99_trip:24-024…","""05:34:34""","""05:34:34""",0,0,0,"""24B-202""","""2025-06-17""","""24-02403-f61cae70""","""CITY CENTRE""","""7149-6d6d1e99""",39,"""New Lynn And Blockhouse Bay To…","""24-02403-18000-2-0d6f00a1""","""05:00:00""",2025-06-17
"""stoptrip""","""stop:7149-6d6d1e99_trip:1278-0…","""05:36:32""","""05:36:32""",0,0,0,"""22R-202""","""2025-06-17""","""1278-02201-e82e4758""","""CITY CENTRE""","""7149-6d6d1e99""",39,"""Rosebank Rd To City Centre Via…","""1278-02201-18120-2-e485a8f4""","""05:02:00""",2025-06-17
"""stoptrip""","""stop:7149-6d6d1e99_trip:1281-0…","""05:37:02""","""05:37:02""",0,0,0,"""25L-202""","""2025-06-17""","""1281-02503-2e5a836a""","""CITY CENTRE""","""7149-6d6d1e99""",41,"""Lynfield To City Centre Via Do…","""1281-02503-18300-2-cbcd1590""","""05:05:00""",2025-06-17
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""stoptrip""","""stop:7149-6d6d1e99_trip:1277-0…","""23:37:32""","""23:37:32""",0,0,0,"""22N-202""","""2025-06-17""","""1277-02203-12b86fa2""","""CITY CENTRE""","""7149-6d6d1e99""",30,"""New Lynn To City Centre Via Ne…","""1277-02203-83220-2-f9fb00ea""","""23:07:00""",2025-06-17
"""stoptrip""","""stop:7149-6d6d1e99_trip:1153-0…","""23:40:58""","""23:40:58""",0,0,0,"""70-205""","""2025-06-17""","""1153-07005-e4111e63""","""BRITOMART""","""7149-6d6d1e99""",39,"""Botany To Britomart Via Panmur…","""1153-07005-82800-2-70d1384b""","""23:00:00""",2025-06-17
"""stoptrip""","""stop:7149-6d6d1e99_trip:1285-0…","""23:41:28""","""23:41:28""",0,0,0,"""75-202""","""2025-06-17""","""1285-07501-22c657d8""","""WYNYARD Q""","""7149-6d6d1e99""",34,"""Glen Innes To Wynyard Quarter …","""1285-07501-83700-2-0770861e""","""23:15:00""",2025-06-17
